In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf

# Import and inspect data

## Import

In [3]:
def import_data():
    """Import training and testing data, as data and labels"""
    
    # Import in pandas (np.genfromtxt is too slow)
    tr_pd = pd.read_csv('data_train.csv', header=None)
    tst_pd = pd.read_csv('data_test.csv', header=None)
    
    # Split data and labels
    tr_data = tr_pd.loc[:, 1:].values
    tr_labels = tr_pd.loc[:, 0].values
    tst_data = tst_pd.loc[:, 1:].values
    tst_labels = tst_pd.loc[:, 0].values
    
    return tr_data, tst_data, tr_labels, tst_labels

In [4]:
def make_onehot(labels):
    """Change numbers 0-9 to unit vectors along the first 10 axes."""
    
    labels_onehot = np.zeros(shape=(labels.shape[0], 10))
    for num in range(labels.shape[0]):
        labels_onehot[num, labels[num]] = 1.0
    
    return labels_onehot

In [5]:
g_tr_data, g_tst_data, g_tr_labels, g_tst_labels = import_data()
g_tr_data = g_tr_data/255
g_tst_data = g_tst_data/255
g_tr_labels_oh = make_onehot(g_tr_labels)
g_tst_labels_oh = make_onehot(g_tst_labels)

## Inspect

In [6]:
def inspect_number(data, labels, imagenum, probs=None):
    """Make a function that displays the image. If a vector of onehot vectors is 
    also inputted, then a bar plot of these probabilities is shown."""
    
    # Reshape the datasets
    plot_data = np.reshape(data, newshape=(data.shape[0], 28, 28))
    
    # If there are no probabilites entered, just plot the picture
    if probs == None:
        print(labels[imagenum])
        fig, ax = plt.subplots()
        ax.imshow(plot_data[imagenum], cmap='Greys')
        plt.show()
    
    # When probabilities are entered, plot the picture and the 
    # probability that it is each number
    else:
        print(labels[imagenum])
        fig, (ax1, ax2) = plt.subplots(1, 2)
        ax1.imshow(plot_data[imagenum], cmap='Greys')
        ax2.bar(np.arange(10), probs)
        asp = np.diff(ax2.get_xlim())[0] / np.diff(ax2.get_ylim())[0]
        ax2.set_aspect(asp)
        plt.show()

# Build MLP in Tensorflow

In [7]:
tf.reset_default_graph()    # Sometimes solves issues 

class MLP():
    def __init__(self, learn_param):
        self.learn_param = learn_param
        
        self.inp = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='inputs')
        self.tgt = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='targets')
        self.sess = tf.Session()
        self._build_graph()
        self.sess.run(tf.initializers.global_variables())
        
    
    def _build_graph(self):
        """Build the neural network"""
        
        initializer = tf.glorot_uniform_initializer()
        
        # The neural network
        h = tf.layers.dense(self.inp, 128, kernel_initializer=initializer, \
                activation=tf.tanh, name='1')
        h = tf.layers.dense(h, 64, kernel_initializer=initializer, \
                activation=tf.tanh, name='2')
        self.logits = tf.layers.dense(h, 10, kernel_initializer=initializer, \
                activation=tf.identity, name='3')
        self.softmax = tf.nn.softmax(self.logits)
        
        # Optimiser: softmax cross entropy
        CE = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.tgt, logits=self.logits)
        self.cost = tf.reduce_mean(CE)
        self.optimizer = tf.train.AdamOptimizer(\
                learning_rate=self.learn_param).minimize(self.cost)
        
        
    def make_minibatch(self, data, labels, batch_size, batch_num):
        """Form a minibatch from the data"""
        
        llim = batch_num * batch_size
        rlim = (batch_num + 1) * batch_size
        return data[llim:rlim], labels[llim:rlim]
    
    
    def evaluate_MLP_outputs(self, data):
        """Run the feedforward NN on some data"""
        
        feed_dict = {self.inp: data}
        outputs = self.sess.run(self.softmax, feed_dict=feed_dict)
        return outputs
    
    
    def calc_frac_correct(self, data, labels):
        """Determine the number of correctly idenfitied pictures across a dataset"""
        
        num_pts = data.shape[0]
        estimates_sm = self.evaluate_MLP_outputs(data)
        estimates = np.argmax(estimates_sm, axis=1)    # Argmax over softmax = estimates
        fraction_correct = 1 - len(np.nonzero(estimates - labels)[0])/num_pts
        return fraction_correct
    
    
    def train_iteration(self, data, labels_oh):
        """Do one training iteration"""
        
        feed_dict = {self.inp: data, self.tgt: labels_oh}
        _, out = self.sess.run([self.optimizer, self.softmax], feed_dict=feed_dict)
        
        
    def train_full(self, tr_data, tr_labels, tst_data, tst_labels, batch_size, num_epochs):
        """Train the neural network and keep track of progress by (cross) validating"""
        
        tr_labels_oh = make_onehot(tr_labels)
        tst_labels_oh = make_onehot(tst_labels)
        
        # batch_size must divide num_tr_points
        num_tr_pts = tr_data.shape[0]
        num_tst_pts = tst_data.shape[0]
        num_batches = int(round(num_tr_pts / batch_size)) 
        
        # Train the MLP and keep track of the errors across training and testing datasets
        batch_nums, epochs, tr_accs, tst_accs = \
                np.array([]), np.array([]), np.array([]), np.array([])
        for epoch in range(num_epochs):
            for batch_num in range(num_batches):
                # Input proportion correctly identified across training and testing dataset
                if batch_num % 50 == 0:
                    tr_acc = self.calc_frac_correct(tr_data, tr_labels)
                    tst_acc = self.calc_frac_correct(tst_data, tst_labels)
                    print('Training accuracy after {} batches: {}'.format(batch_num, tr_acc))
                    epochs = np.append(epochs, epoch)
                    batch_nums = np.append(batch_nums, batch_num)
                    tr_accs = np.append(tr_accs, tr_acc)
                    tst_accs = np.append(tst_accs, tst_acc)
                
                # Do the training
                mb_data, mb_labels_oh = self.make_minibatch(tr_data, tr_labels_oh, \
                        batch_size=batch_size, batch_num=batch_num)    # Make minibatch
                self.train_iteration(mb_data, mb_labels_oh)    # Do single training iteration
                
        accuracies = np.vstack((epochs, batch_nums, tr_accs, tst_accs)).T
        return accuracies

In [14]:
tf.reset_default_graph()    # Sometimes solves issues
def train_MLP_NN():
    model = MLP(learn_param=0.01)
#    accuracies = model.train_full(g_tr_data, g_tr_labels, g_tst_data, g_tst_labels, 100, 2)
#    accuracies = pd.DataFrame(accuracies, columns = ['epoch', 'batch', 'train', 'test'])
#    return accuracies
train_MLP_NN()

In [15]:
tf.global_variables()

[<tf.Variable '1/kernel:0' shape=(784, 128) dtype=float32_ref>,
 <tf.Variable '1/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable '2/kernel:0' shape=(128, 64) dtype=float32_ref>,
 <tf.Variable '2/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable '3/kernel:0' shape=(64, 10) dtype=float32_ref>,
 <tf.Variable '3/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable '1/kernel/Adam:0' shape=(784, 128) dtype=float32_ref>,
 <tf.Variable '1/kernel/Adam_1:0' shape=(784, 128) dtype=float32_ref>,
 <tf.Variable '1/bias/Adam:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable '1/bias/Adam_1:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable '2/kernel/Adam:0' shape=(128, 64) dtype=float32_ref>,
 <tf.Variable '2/kernel/Adam_1:0' shape=(128, 64) dtype=float32_ref>,
 <tf.Variable '2/bias/Adam:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable '2/bias/Adam_1:0' shape=(64,) dtype=flo

In [10]:
train_MLP_NN()

ValueError: Variable 1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-7-a9ba66ab60c2>", line 21, in _build_graph
    activation=tf.tanh, name='1')
  File "<ipython-input-7-a9ba66ab60c2>", line 10, in __init__
    self._build_graph()
  File "<ipython-input-8-d3a9e7d78151>", line 3, in train_MLP_NN
    model = MLP(learn_param=0.01)


In [ ]:
"""Calculate number of variables"""

tf.global_variables()

In [ ]:
"""Important results:

                        1 hidden   2 hidden
                        layer      layers
                        
Number of parameters:   14         20
Time taken (5 epochs):  172s       223s
Final training error:   0.971      0.956
Final testing error:    0.962      0.948

"""

In [ ]:
"""Plot training and testing errors"""

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
errors_1HL = pd.read_csv('1HL.csv', index_col=[0])
errors_2HL = pd.read_csv('2HL.csv', index_col=[0])

X = (np.arange(60) + 1) / 12
tr_1HL = errors_1HL.loc[:, 'training']
tst_1HL = errors_1HL.loc[:, 'testing']
tr_2HL = errors_2HL.loc[:, 'training']
tst_2HL = errors_2HL.loc[:, 'testing']
ax1.plot(X, tr_1HL)
ax1.plot(X, tst_1HL)
ax2.plot(X, tr_2HL)
ax2.plot(X, tst_2HL)
ax1.set_xlabel('Runs through dataset')
ax2.set_xlabel('Runs through dataset')
ax1.set_ylabel('Proportion correct')
ax1.set_title('1 Hidden layer')
ax2.set_title('2 Hidden layers')
ax1.legend()
ax2.legend()
plt.savefig('Errors')